# Test the edge configuration package locally
This notebook shows how to use `LocalPipelineRunner` to test the components of the edge package in a local virtual Python environment.

The edge package shown and used was created with the [30-CreatePipelinePackage.ipynb](30-CreatePipelinePackage.ipynb) notebook.

Please note, there is no official TFLite runtime for Windows.

You can either use the full TensorFlow package, or build the TFLite wheel manually, following this guide: https://www.tensorflow.org/lite/guide/build_cmake_pip

#### Imports

In [ ]:
import glob
import os
from pathlib import Path
import pandas as pd
import sys

from simaticai.testing.pipeline_runner import LocalPipelineRunner

#### Create image input from a JPEG or PNG file

In [ ]:
sys.path.append('../src')
from payload import create_imageset_dict

Create the required input format from an input JPG file using the method defined above.

In [ ]:
payload = create_imageset_dict('../data/processed/simatic_photos/S7_1500/IMG_1119.JPG', "BayerRG8")
pipeline_input1 = { "vision_payload": payload }

#### Test the edge configuration package locally
Take the already created image package to be tested as input of the `LocalPipelineRunner`, and run the pipeline with the test data.
The `LocalPipelineRunner` module is designed to be used __only__ inside a `with` block.

The `run_pipeline` method can be called multiple times but be aware that the internal state of the components is reset between the calls. Use batch input if you want to keep the state between inputs.

In [ ]:
# Adjust the version number as needed
image_on_edge_package = Path('../packages/ImageClassification-edge_1.zip')

os.environ["LOGLEVEL"] = "INFO"  # for configuring the LOGLEVEL of log module, it is necessary to set the LOGLEVEL environment variable to the required level.

test_dir = Path.home() / "test"

with LocalPipelineRunner(image_on_edge_package, test_dir) as runner:
    pipeline_output = runner.run_pipeline([pipeline_input1])

Examine output.

In [ ]:
pipeline_output

#### Test the components locally
Alternatively, you can run the components separately if you want to inspect their outputs.

Remember: the internal state of the components is reset after `run_component` returns. Use batch input if you want to keep the state between successive inputs.

In [ ]:
os.environ["LOGLEVEL"] = "DEBUG"    # by setting LOGLEVEL to DEBUG, you can get more detailed logs from log module. This is set by default.
inference_component_input = [pipeline_input1]
with LocalPipelineRunner(image_on_edge_package, test_dir) as runner:
    inference_component_output = runner.run_component('inference', inference_component_input)

Examine output.

In [ ]:
inference_component_output

#### Feed the complete directory of images through pipeline

As the saved model will contain no information about what the numerical classes mean, we collect the class labels in a list.  
Labels correspond to directories.

In [ ]:
image_dir = Path('../data/processed/simatic_photos')
labels = [path.name for path in image_dir.iterdir() if path.is_dir()]
labels.sort()
labels

Pick a directory and generate pipeline input from the JPG files in it.

In [ ]:
directory = labels[0]
data_path = Path("../data/processed/simatic_photos") / directory
files = glob.glob(f"{data_path}/*.JPG")
pipeline_input = [{ "vision_payload": create_imageset_dict(image_file, "BayerRG8")} for image_file in files]
print(f"{len(pipeline_input)} file(s) in {data_path}")

Send the input through the pipeline and capture output.

In [ ]:
os.environ["LOGLEVEL"] = "INFO"

with LocalPipelineRunner(image_on_edge_package, test_dir) as runner:
    pipeline_output = runner.run_pipeline(pipeline_input)

Examine the distribution of predictions. The majority of predictions should be in one of the classes.

In [ ]:
predictions = pd.DataFrame([output["prediction"] for output in pipeline_output])
predictions.value_counts()

#### Manually recorded test results for different directories

For the example images, we recorded a result for every class into the next list of lists which represents the predicted classes for the given directories.
You can check the results for the 'ET200AL' images in the first row as this label is stored in the 0 position of the list.

In [ ]:
results = [[167,54,11,21,0],
           [15,197, 2,10,2],
           [13,12,150,18,0],
           [0, 2, 2,107, 5],
           [4, 1,1,22,135]]

You also can visualize this kind of confusion matrix with our helper method as shown below.

In [ ]:
import numpy
from utils import show_confusion_matrix

confusion_matrix = numpy.array([numpy.array(l) for l in results])
show_confusion_matrix(confusion_matrix, labels)